# Exploring the data.seattle.gov API
The API for data.seattle.gov is called the SODA API (for "Socrata Open Data API"). This API allows you to do a lot of filtering and selecting your data before you download it. Not all APIs offer this kind of expanded 'query' functionality, but it can be very useful when they do, for at least 2 reasons:

1. it is often easier to select and filter the data you want with an API query than by using Python loops and conditional statements.
2. it allows you to download only a subset of the dataset. If the dataset is very large this can save space on your computer and speed up your code (because Python doesn't have to loop through so many rows every time you perform an operation).

In this notebook, we will walk through a couple of examples of different kinds of API queries you can do with the data.seattle.gov API. If you are interested in getting more practice with this API, you can check out [the API docs here](https://dev.socrata.com/). 

# Step 1: test the API (do this first)
Run the code in the cell below. You should a list with one large dictionary inside it, which contains a bunch of information about the fancy new [Bill and Melinda Gates Computer Science building](https://www.engr.washington.edu/about/bldgs/cse2).

In [ ]:
# import csv
# import json
import requests
from pprint import pprint

# save the API endpoint. This is the base URL for all API requests for the building permit dataset
test_endpoint = "https://data.seattle.gov/resource/76t5-zqzr.json?"

#specify some parameters (filtering the data so we only get the parts we want)
test_params = "$where=(PermitNum='6531736-PH')"

test_url = test_endpoint + test_params

test_request = requests.get(test_url)
test_data = test_request.json()

pprint(test_url)
pprint(test_data)

## What just happened?
You just made an APi request (or "query") to data.seattle.gov, and requested a single item (or "row") from the [dataset of Seattle building permits](https://data.seattle.gov/Permitting/Building-Permits/76t5-zqzr). This dataset contains information about 135,000 proposed and completed building projects within the city of Seattle, going back at least a decade.

In this case, we specified in our API request that we wanted data on a single building, using the parameter ("filtering criteria") ``$where=(PermitNum='6531736-PH')``. Since each row in this dataset has a unique permit number, we were able to retrieve a single record from the entire dataset.

## Using more complex parameters in your API query
In most cases where you are gathering data from an API you will want to retrieve more than one item, and/or you won't know the unique id for the record(s) you want to retrieve. 

In these cases, you can use multiple parameters in combination to grab a set of rows from the dataset that share some characteristics.

Some useful parameters are:
- ``select``: which columns from the dataset you want to get for each item or row
- ``where``: specific values for a column that you want (or don't want)
- ``order``: sort by one of the columns in the output, e.g. start with the most recently issued permits
- ``limit``: the maximum number of many rows you want to get


### Finding column and value names in a dataset
In many cases where you want to download data from a website via an API, you won't be able to see what the data looks like (its structure and its content) before you start querying. Fortunately, data.seattle.gov has [a spreadsheet-like interface](https://data.seattle.gov/Permitting/Building-Permits/76t5-zqzr/data) for each dataset, where you can preview the data and see the names of all of the columns. This makes it easier for us to decide how to build our query.

## Creating a more complex API query

For example, anyone who lives in central Seattle knows that there have been a LOT of new apartment and condo buildings going up over the past decade! Let's say we wanted to track the rate of apartment building creation in Seattle over time, to see if the rate really has increased.

Using a combination of different parameters in a single query, we can filter the building permit dataset and retrieve only the records we want.

When doing a complex query like this, it is often useful to write it out for yourself in human language first, and then translate that into API parameters (the language that the computer reads).

"Give me basic information for the 1000 most recently completed new single and multifamily dwellings built in Seattle since 2006"

This query would look like this (paste it into your browser to try it out!):
``https://data.seattle.gov/resource/76t5-zqzr.json?$select=PermitNum, PermitClass, CompletedDate, Description&$where=(PermitClass='Multifamily' OR PermitClass='Single Family/Duplex') AND PermitTypeDesc='New' AND CompletedDate > '2006-01-01'&$order=CompletedDate DESC&$limit=1000``

As you can see, these URLs can get long and complicated! So it is often easier to send these complex queries using the ``params`` argument of the ``requests`` library. This allows you to organize these sub-components fo the query into a dictionary, where the keys are the parameters and the values are the filtering criteria.

In [ ]:
bp_api_params = {
"$select" : "PermitNum, PermitClass, CompletedDate, Description",
"$where" : "(PermitClass='Multifamily' OR PermitClass='Single Family/Duplex') AND PermitTypeDesc='New' AND CompletedDate > '2006-01-01'",
"$order" : "CompletedDate DESC",
"$limit" : "1000",
}

Then you can pass this dictionary, along with the endpoint (see the first cell above) into the request.

In [ ]:
bp_endpoint = "https://data.seattle.gov/resource/76t5-zqzr.json?"

bp_request = requests.get(bp_endpoint, params=bp_api_params)
bp_data = bp_request.json()

pprint(bp_request.url) #you can still print out the full URL you sent to the API, to help debug
pprint(bp_data[:5])#print out the first 5 items to check our work

## Saving to CSV
Now that you have the data you want stored in the list-of-dictionaries called ``bp_data``, you can start to work with it in Python! Or, if you just want to save this data file and work on it in a spreadsheet, you can save it directly to CSV at this point.

In [ ]:
 
#read through the list-of-dicts line by line and write it to CSV    
with open('1000_most_recent_residential_permits.csv', 'w') as csv_outfile:
    writer = csv.writer(csv_outfile)

    #first write the titles that will appear at the head of each column in the CSV
    writer.writerow(('permit number', 'description', 'building type', 'completed date'))
    
    for b in bp_data:

        #write the data for each permit application onto a single row in the CSV 
        writer.writerow((b['PermitNum'], b['Description'], b['PermitClass'], b['CompletedDate']))

## Note on messy data
The data in this dataset is messy and incomplete! Some of the rows in this dataset will be missing values for fields (like "Original Zipcode" and "HousingUnitsAdded" to name a few). You may find that this leads to errors when you are trying to analyze or export the data. 

In general, dealing with messy, inconsistent, incomplete data is the MOST TIME CONSUMING (and frustrating!) part of doing data science. If you are trying to write your own query and you are getting errors that you suspect may be due to missing or messy data, call a mentor over and we can work with you to try to debug this.